This notebook was prepared by [SentinelWarren](https://github.com/sentinelwarren). Source and license info is on [GitHub](https://github.com/SentinelWarren/patterns-in-ools).

# Singleton Pattern

## Description

Is a way to provide one and only one object of a particular type. The Singleton class can have only one instance, and provides global access to that one instance.

Due to its instantiation restrictions of a class to one object, this pattern is useful when you need one object to coordinate actions for the system.

The basic idea is that only one instance of a particular class, doing a job, is created for the needs of the program. To ensure that this works, we need mechanisms that prevent the instantiation of the class more than once and also prevent cloning.

## Singleton in the wild

Singleton pattern can be used to implement projects that do not affect the execution of code, hence its acceptable to use global state. The kinds of projects are;

* Logging
* Caching
* Load balancing
* Route mapping
* A single Database object

In all these cases,information flows in one direction, and the singleton instance itself is immutable (it does not change).
No part of the program attempts to make a change in the singleton, and as such there is no danger of one part of a project interfering with another part of the project because of the shared state.

### Example projects

* Plone CMS

## Pros & Cons

### Pros

* Access to global state, making a Singleton instance to be immutable.
* The singleton object is initialized only when it’s requested for the first time.
* Assurance that a class has only a single instance.

### Cons | Drawbacks

* Violates the Single Responsibility Principle
* Possibility of coupling your code to itself in very unfortunate ways, causing unexpected results in completely unrelated piece of code.
* The pattern requires special treatment in a multithreaded environment so that multiple threads won’t create a singleton object several times.

## Use cases

* Maintaining a global state of a program

* Controlling concurrent access to a shared resource. For example, the class managing the connection to a database.

* A service or resource that is transversal in the sense that it can be accessed from different parts of the application or by different users and do its work. For example, the class at the core of the logging system or utility.

## Implementation

* Using metaclass and its type, having predefined the `Singleton` metaclass.
* As required, the metaclass's `__call__()` method holds the code that ensures that only one instance of the class can be created.

### Test cases

#### Unit

* `len(SingletonClass().instances) == 0`
* `SingletonClass() == SingletonClass()`
* `SingletonClass().attributes == SingletonClass().attributes`
* `len(SingletonClass().instances) == 1`
* `len(SingletonClass().instances) >= 1`

#### Functional

* `Singleton().instances['invalid'] raise KeyError`

In [70]:
%%writefile singleton.py
"""Singleton Pattern sample code.

Intent: Lets you ensure that a class has only one instance, while providing a
global access point to this instance. One instance per each subclass (if any).
"""
from typing import Dict


class SingletonMeta(type):
    """Singleton metaclass
    
    The Singleton class can be implemented in different ways in Python.
    Possible methods include: base class, decorator, metaclass.
    """
    _instances: Dict[str, object] = {}

    def __call__(cls, *args: object, **kwargs: object) -> object:
        if str(cls) not in cls._instances:
            cls._instances[str(cls)] = super(SingletonMeta,
                                             cls).__call__(*args, **kwargs)

        return cls._instances[str(cls)]

    @staticmethod
    def get_instances():
        """Returns SingletonMeta _instances"""
        return SingletonMeta._instances


class FooTone(metaclass=SingletonMeta):
    """Client class"""

    def __init__(self, arg: str) -> None:
        self.attribute = arg

    def some_method(self):
        """Some method implemanting business logic,
        which can be executed on its instance."""

    def __repr__(self) -> str:
        return (
            f"<class {self.__class__.__name__}("
            f"{self.attribute!r}) "
            f"at {hex(id(self))}>"
        )


Overwriting singleton.py


## Tests

Using `pytest`

In [66]:
%%writefile ../../tests/unit/test_singleton.py
"""Testing singleton module"""
from pypatterns import singleton, logging


singleton_instances = singleton.SingletonMeta.get_instances()
FOO_CLASS = "<class 'pypatterns.creational.singleton.FooTone'>"
LOGGER_CLASS = "<class 'pypatterns.examples.creational.logging.Logger'>"

def instantiate_foo(arg):
    """Instantiate singleton.FooTone class"""
    return singleton.FooTone(arg)

def instantiate_logger(arg):
    """Instantiate logging.Logger class"""
    return logging.Logger(arg)

def delete_instance(name):
    """Delete an instance in singleton.SingletonMeta"""
    if name in singleton_instances:
        del singleton_instances[name]

def test_global_state():
    """Test Singleton global state"""
    foo1 = instantiate_foo("some_attr")
    foo2 = instantiate_foo("diff_attr")
    assert foo1 is foo2
    assert foo1.attribute == foo2.attribute

    foo2.attribute = "another_attr"
    assert foo1.attribute == foo2.attribute

def test_empty_instances():
    """Test empty singleton.Singleton._instances"""
    singleton_instances.clear()

    assert len(singleton_instances) == 0
    assert LOGGER_CLASS, FOO_CLASS not in singleton_instances

def test_all_added_instances():
    """Test all added instances in singleton.Singleton._instances"""
    if LOGGER_CLASS not in singleton_instances:
        instantiate_logger("file.log")

    if FOO_CLASS not in singleton_instances:
        instantiate_foo("some_attr")

    assert len(singleton_instances) == 2
    assert FOO_CLASS, LOGGER_CLASS in singleton_instances

def test_logger_instance():
    """Test only logging.Logger instance in singleton.Singleton._instances"""
    delete_instance(FOO_CLASS)

    assert len(singleton_instances) == 1
    assert LOGGER_CLASS in singleton_instances

def test_foo_instance():
    """Test only singleton.FooTone instance in singleton.Singleton._instances"""
    instantiate_foo("some_attr")
    delete_instance(LOGGER_CLASS)

    assert len(singleton_instances) == 1
    assert FOO_CLASS in singleton_instances

Overwriting ../../tests/unit/test_singleton.py


In [67]:
%%writefile ../../tests/func/test_singleton_exceptions.py
"""Testing singleton.SingletonMeta exceptions"""
from pypatterns import singleton
from pytest import raises

def test_instances_raises():
    """Test KeyError exception"""
    with raises(KeyError):
        instances = singleton.SingletonMeta.get_instances()
        instances['invalid-key']

Overwriting ../../tests/func/test_singleton_exceptions.py


In [1]:
%run -m pytest ../../tests/unit/test_singleton.py ../../tests/func/test_singleton_exceptions.py

......                                                                   [100%]
6 passed in 0.06s


# Project implementation

A simple project implementation using the Singleton pattern.

## `logger.py`

A simple logging module that keeps track of the `.log` file state, writes specific logging messages to the log file accordingly, and reads the contents of the log file.

In [68]:
%%writefile ../examples/creational/logging.py
"""Example implementation of Singleton Pattern"""
from typing import Any
from pypatterns import singleton


class Logger(metaclass=singleton.SingletonMeta):
    """Logger class using Singleton pattern"""
    def __init__(self, file_name: str) -> None:
        self.file_name = file_name

    def _write_log(self, level: str, msg: Any) -> None:
        """Write a logging message depending on the level
        to the global log file"""
        with open(self.file_name, "a", encoding='utf-8') as log_file:
            log_file.write(f"[{level}] {msg}\n")

    def critical(self, msg: Any) -> None:
        """Log message of level critical"""
        self._write_log('CRITICAL', msg)
    
    def error(self, msg: Any) -> None:
        """Log message of level error"""
        self._write_log('ERROR', msg)

    def warn(self, msg: Any) -> None:
        """Log message of level warning"""
        self._write_log('WARNING', msg)

    def info(self, msg: Any) -> None:
        """Log message of level info"""
        self._write_log('INFO', msg)

    def debug(self, msg: Any) -> None:
        """Log message of level debug"""
        self._write_log('DEBUG', msg)

    def read_log(self) -> str:
        """Read the global log file"""
        with open(self.file_name, "r", encoding='utf-8') as log_file:
            return log_file.read()

    def print_log(self) -> None:
        """Print the log output to the console"""
        print(self.read_log())

    def __repr__(self) -> str:
        return (
            f"<class {self.__class__.__name__}("
            f"{self.file_name!r}) "
            f"at {hex(id(self))}>"
        )



if __name__ == '__main__':
    logger = Logger("class_logger.log")
    logger.warn('AE-35 hardware failure predicted!')
    logger.print_log()
    

Overwriting ../examples/creational/logging.py


## Functional Test

Using `pytest`

In [69]:
%%writefile ../../tests/func/test_logger.py
"""Testing logging.Logger functionality"""
from pypatterns import logging


def test_methods_return(tmpdir_factory):
    """Test logging.Logger methods return"""
    tmp = tmpdir_factory.mktemp("func")
    log = logging.Logger(tmp.join("logger.log"))
    expected = None

    assert log.critical("This is a critical message") == expected
    assert log.error("This is an error message") == expected
    assert log.warn("This is a warning message") == expected
    assert log.info("This is an info message") == expected
    assert log.debug("This is a debugging message") == expected
    assert log.print_log() == expected

def test_log_output():
    """Test log file output"""
    expected_log = (
        "[CRITICAL] This is a critical message\n"
        "[ERROR] This is an error message\n"
        "[WARNING] This is a warning message\n"
        "[INFO] This is an info message\n"
        "[DEBUG] This is a debugging message\n"
    )
    assert logging.Logger("logger.log").read_log() == expected_log

Overwriting ../../tests/func/test_logger.py


In [1]:
%run -m pytest ../../tests/func/test_logger.py

..                                                                       [100%]
2 passed in 0.03s
